# Exploration
We will first need to have playwright initialize itself so that we can programmatically crawl a website to automatically gather the reviews

In [2]:
!playwright install


BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu20.04-x64.
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu20.04-x64.
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu20.04-x64.
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu20.04-x64.
Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libicudata.so.66                                 ║
║     libicui18n.so.66                                 ║
║     libicuuc.so.66                                   ║
║     libjpeg.so.8                                     ║
║     libwebp.so.6                                     ║
║     libpcre.so.3                                     ║
║     libffi.so.7     

## Using Playwright within a jupyter notebook
Playwright will usually work in python scripts. To explore the pages, we'll need to use asyncio. You can view the full explanation in this stack overflow answer ["How to use the Playwright library in a Jupyter notebook instead of using a regular .py script (on Windows)"](https://stackoverflow.com/a/71702599)

In [3]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

await page.goto("http://whatsmyuseragent.org/")

<Response url='http://ww1.whatsmyuseragent.org/?usid=17&utid=34089031374' request=<Request url='http://ww1.whatsmyuseragent.org/?usid=17&utid=34089031374' method='GET'>>

## Accessing the website

I have chosen to collect reviews for LTT Store, as I follow their youtube and find their products to be quality. LTT Store used to have reviews embedded in the product page, but when I was drafting this notebook, it wasn't accessible. Fortunately, I had some old emails, and I was able to trace it back to judge.me. To keep it simple, let's open the backpack reviews [link](https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack)

In [10]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.firefox.launch(headless=False)
page = await browser.new_page()

# Set the viewport size
await page.set_viewport_size({"width": 1920, "height": 1080})

await page.goto("https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack")


<Response url='https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack' request=<Request url='https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack' method='GET'>>

You should see a window that looks like this
![](../dev-log/supporting_files/images/2024-08-18/20240818163226.png)
With the website loaded, we'll now want to filter the reviews by most recent first. To access this, we will need to inspect the elements and review the html. Within the playwright window, right click anywhere on the page.

In [62]:
await page.click("select.filter-sort__sort-dropdown")

# Wait for the options to be visible and then select 'Most Recent'
await page.select_option("select.filter-sort__sort-dropdown", label="Most Recent")

['[object Object]']

In [59]:
dropdown = page.locator('role=combobox')
await dropdown.click()
# Use get_by_role to select the 'option' and then choose 'Most Recent'
await dropdown_locator.select_option(label="Most Recent")

TargetClosedError: Locator.select_option: Target page, context or browser has been closed

In [64]:

    # Set up the response handler to log all responses
    def log_response(response):
        print("Response received:", response.url(), response.status())
    page.on("response", log_response)

    # Set the viewport size
    await page.set_viewport_size({"width": 1920, "height": 1080})

    # Navigate to the specific page
    await page.goto("https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack")

    # Handling dropdown interaction
    dropdown = page.locator('role=combobox')
    await dropdown.click()
    await dropdown.select_option(label="Most Recent")  # Assuming this triggers the API request

    # Wait for a specific response to finish
    response = await page.wait_for_response(lambda r: 'judge.me/api/v1/widgets/product_review' in r.url() and r.status() == 200)

    # Optionally, process the response
    json_body = await response.json()
    print("Specific response content:", json_body)

    # Keep the browser open long enough to observe manually or further automate
    await asyncio.sleep(10)  # Wait for 10 seconds

    # Close the browser
    # await browser.close()

TypeError: Locator.click: 'str' object is not callable

In [58]:
test = []

In [59]:
test

[]

In [60]:
    playwright = await async_playwright().start()
    browser = await playwright.firefox.launch(headless=False)
    page = await browser.new_page()

    # Set the viewport size
    await page.set_viewport_size({"width": 1920, "height": 1080})

    # https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=1&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&review_type=product-reviews&review_source=all_sources&verified_only=false

    # Listen for specific responses
    async def handle_response(response):
        url = response.url
        if "judge.me/api/v1/widgets/product_review" in url and 'sort_by=created_at' in url:
            print("Response received from:", url)
            print("Content-Type:", response.headers['content-type'])
            try:
                json_body = await response.json()
                print("Specific response content:", json_body)
                test.append(json_body)
            except Exception as e:
                print("Error processing JSON:", e)

    # Attach the response handler to the page
    page.on("response", handle_response)

    # Navigate to the specific page
    await page.goto("https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack")

    # Wait for the dropdown to be visible and interact with it
    dropdown = page.locator('role=combobox')
    await dropdown.wait_for(state="visible")
    await dropdown.click()

    # Select the option that triggers the network request
    await dropdown.select_option(label="Most Recent")

    # Optionally, hold the browser open a bit longer to ensure all responses are captured
    await asyncio.sleep(10)  # Adjust the sleep duration as necessary

Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=1&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&review_type=product-reviews&review_source=all_sources&verified_only=false
Content-Type: application/json; charset=utf-8
Specific response content: {'reviews': [{'uuid': 'b8f0ba8d-2fa5-4261-b48e-b6be38b4fcdb', 'title': 'A very nice backpack', 'rating': 4, 'body_html': "<p>Overall, the best backpack I've owned and will continue to use it well into the future. </p>\n\n<p>Pros:\n<br />Feels very sturdy and durable </p>\n\n<p>Lots of storage space </p>\n\n<p>Straps are quite comfortable </p>\n\n<p>Design is practical and well thought-out </p>\n\n<p>Water-resistance has been excellent so far</p>\n\n<p>Cons:\n<br />The foam shoulder padding on the straps seems to be wearing out very unevenly. I always have the backpack over bot

Response received from: https://judge.me/api/v1/widgets/product_review?platform=shopify&shop_domain=linus-tech-tips-store.myshopify.com&api_token=snmsgMp9hQgfed2xliHdvcJig1o&external_id=6649895256167&page=2&sort_by=created_at&sort_dir=desc&json_request=true&per_page=10&search=&review_type=product-reviews&review_source=all_sources&verified_only=false
Content-Type: application/json; charset=utf-8
Specific response content: {'reviews': [{'uuid': 'ee3f743b-ab7e-4051-b6b4-c67d97e90fa0', 'title': 'Disappointed in backpack', 'rating': 2, 'body_html': '<p>On paper, this backpack reads and looks terrific and I purchased it as a birthday gift for my grandson (upon his advice). Unfortunately, after two outings, he found that the backpack "rode" on his shoulders in such a way as to cause both chaffing and discomfort.\n<br />He LOVES the concept of this backpack (lots of room, plan of backpack is superior to others)  and has tried to make adjustments to \n<br />how he wears it but has not been succ

In [63]:
test

[{'reviews': [{'uuid': 'b8f0ba8d-2fa5-4261-b48e-b6be38b4fcdb',
    'title': 'A very nice backpack',
    'rating': 4,
    'body_html': "<p>Overall, the best backpack I've owned and will continue to use it well into the future. </p>\n\n<p>Pros:\n<br />Feels very sturdy and durable </p>\n\n<p>Lots of storage space </p>\n\n<p>Straps are quite comfortable </p>\n\n<p>Design is practical and well thought-out </p>\n\n<p>Water-resistance has been excellent so far</p>\n\n<p>Cons:\n<br />The foam shoulder padding on the straps seems to be wearing out very unevenly. I always have the backpack over both shoulders, not sure why the left strap would flatten out while the right still looks new.\n<br />A bit heavy, even while empty</p>\n\n<p>Description advertised it stands up on its own but, I haven't found that to be reliable </p>\n\n<p>I wish it had some kind of reflective spots on the back for cyclists/scooter riders at night</p>\n\n<p>For my use case, a slightly bigger main-area, and a smaller sle

In [46]:
# hardcoded next page
next_page_button = page.locator("span.pagination__page-number:nth-child(11)")
        
# Wait for the button to be visible
await next_page_button.wait_for(state="visible")
        
# Click the 'Next Page' button
await next_page_button.click()

In [62]:
# Locate the 'Next Page' button by finding the span with the chevron_right icon
next_page_button = page.locator('span.pagination__page-number--icon:has-text("chevron_right")')

# Wait for the button to be visible
await next_page_button.wait_for(state="visible")

# Click the 'Next Page' button
await next_page_button.click()

# As a script

In [ ]:
import asyncio
import json
from playwright.async_api import async_playwright

async def handle_response(response, page_url, output_file):
    if "judge.me/api/v1/widgets/product_review" in response.url and 'sort_by=created_at' in response.url:
        print("Response received from:", response.url)
        print("Content-Type:", response.headers['content-type'])
        try:
            json_body = await response.json()
            print("Specific response content:", json_body)
            
            # Prepare the new reviews with the current page URL as the key
            new_reviews = {page_url: json_body['reviews']}
            
            # Dump only the new reviews into a JSON file
            with open(output_file, 'a') as f:
                json.dump(new_reviews, f)
                f.write('\n')  # Add a newline for each JSON entry
        except Exception as e:
            print("Error processing JSON:", e)

async def main():
    async with async_playwright() as playwright:
        browser = await playwright.firefox.launch(headless=False)
        page = await browser.new_page()
        
        output_file = "collected_reviews.json"

        # Set the viewport size
        await page.set_viewport_size({"width": 1920, "height": 1080})

        # Navigate to the specific page
        page_url = "https://judge.me/reviews/linus-tech-tips-store.myshopify.com/products/backpack"
        await page.goto(page_url)

        # Attach the response handler to the page
        page.on("response", lambda response: handle_response(response, page_url, output_file))

        # Wait for the dropdown to be visible and interact with it
        dropdown = page.locator('role=combobox')
        await dropdown.wait_for(state="visible")
        await dropdown.click()

        # Select the option that triggers the network request
        await dropdown.select_option(label="Most Recent")

        # Locate the 'Next Page' button by finding the span with the chevron_right icon
        next_page_button = page.locator('span.pagination__page-number--icon:has-text("chevron_right")')

        # Wait for the button to be visible
        await next_page_button.wait_for(state="visible")

        # Click the 'Next Page' button
        await next_page_button.click()

        await browser.close()

# Run the main function
await main()